<a href="https://colab.research.google.com/github/kategavrishina/compling21/blob/main/homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Важное требование ко всей домашке в целом: в jupyter ноутбуке не должно был лишнего кода (т.е. если вы взяли за основу семинар, не забудьте удалить все лишнее)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

from IPython.display import Image
from IPython.core.display import HTML 
%matplotlib inline

In [2]:
! pip install --upgrade gensim

In [3]:
import gensim

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 

1.   добавьте лемматизацию в предобработку (любым способом) 
2.   измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window не так используется)
3. измените часть с np.random.randint(vocab_size) так, чтобы случайные негативные примеры выбирались обратно пропорционально частотностям слов (частотные должны выбираться реже, а редкие чаще)

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

In [4]:
from tqdm import tqdm

In [5]:
! pip install pymorphy2

In [6]:
import pymorphy2

In [7]:
morph = pymorphy2.MorphAnalyzer()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
wiki = open('drive/MyDrive/wiki_data.txt').read().split('\n')

In [10]:
wiki = wiki[:10000]

In [11]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

In [12]:
texts = [preprocess(text) for text in tqdm(wiki)]

100%|██████████| 10000/10000 [10:49<00:00, 15.40it/s]


In [13]:
vocab = Counter()

for text in tqdm(texts):
    vocab.update(text)

100%|██████████| 10000/10000 [00:00<00:00, 17989.50it/s]


In [14]:
filtered_vocab = {}

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab[word] = vocab[word]

In [15]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
id2word = {i:word for word, i in word2id.items()}

In [16]:
sentences = []

for text in tqdm(texts):
    ids = [word2id[token] for token in text if token in word2id]
    sentences.append(ids)

100%|██████████| 10000/10000 [00:00<00:00, 20286.05it/s]


### Skip gram

In [17]:
vocab_size = len(id2word)

counts = np.array(list(filtered_vocab.values()))
probs = 1 - (counts / counts.sum())
rev_probs = probs / probs.sum()
list_ids = [word2id[word] for word in filtered_vocab.keys()]

new_vocab = np.random.choice(list_ids, size=1000000,  p=rev_probs)

In [18]:
def gen_batches_sg(sentences, window=12, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-int(window/2)):i] + sent[i+1:i+int(window/2)]
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)
                    
                    X_target.append(word)
                    X_context.append(np.random.choice(new_vocab))

                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [19]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(1,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)

dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(gen_batches_sg(sentences[:9000]),
          validation_data=gen_batches_sg(sentences[9000:]),
          batch_size=1000,
          steps_per_epoch=10000,
          validation_steps=30,
         epochs=2)

Epoch 1/2
10000/10000 [==============================] - 69s 7ms/step - loss: 0.4623 - accuracy: 0.7879 - val_loss: 0.4536 - val_accuracy: 0.7968
Epoch 2/2
10000/10000 [==============================] - 68s 7ms/step - loss: 0.4503 - accuracy: 0.7975 - val_loss: 0.4799 - val_accuracy: 0.7777


In [21]:
embeddings_sg = model.layers[2].get_weights()[0]

In [22]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in 
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [23]:
most_similar('больница', embeddings_sg)

['больница',
 'магазин',
 'разместиться',
 'сад',
 'клиника',
 'баня',
 'наслать',
 'начальный',
 'бытовой',
 'часовня']

In [24]:
most_similar('животное', embeddings_sg)

['животное',
 'популяция',
 'человеческий',
 'пища',
 'обычно',
 'питаться',
 'растительный',
 'существо',
 'плотный',
 'климат']

Мне кажется, это не очень хороший результат, потому что для некоторых слов я даже не могу придумать быстро обоснование (типа начальный и наслать). Для животного ближайшие слова выглядят получше, вроде бы все они имеют отношение к животным

### CBOW

In [25]:
def gen_batches_cbow(sentences, window=6, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-int(window/2)):i] + sent[i+1:i+int(window/2)]

                X_target.append(word)
                X_context.append(context)
                y.append(1)
                
                X_target.append(np.random.choice(new_vocab))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window+1)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [26]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model.fit(gen_batches_cbow(sentences[:9000]),
          validation_data=gen_batches_cbow(sentences[9000:]),
          batch_size=1000,
          steps_per_epoch=5000,
          validation_steps=30,
         epochs=2)

Epoch 1/2
5000/5000 [==============================] - 73s 14ms/step - loss: 0.3671 - accuracy: 0.8386 - val_loss: 0.3303 - val_accuracy: 0.8596
Epoch 2/2
5000/5000 [==============================] - 72s 14ms/step - loss: 0.2455 - accuracy: 0.8978 - val_loss: 0.3119 - val_accuracy: 0.8746


In [28]:
embeddings_cbow = model.layers[2].get_weights()[0]

In [29]:
most_similar('больница', embeddings_cbow)

['больница',
 'училище',
 'гимназия',
 'мастерская',
 'контора',
 'клиника',
 'госпиталь',
 'колледж',
 'обучаться',
 'мечеть']

In [30]:
most_similar('животное', embeddings_cbow)

['животное',
 'мясо',
 'растение',
 'собака',
 'корова',
 'птица',
 'вещество',
 'рыба',
 'предмет',
 'эксперимент']

Здесь мне результат для больницы кажется более качественным, подобраны разнообразные заведения, среди которых госпиталь, которого не было в Skip Gram. Примеры для животного более частные, конкретные, что тоже неплохо

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [31]:
w2v = gensim.models.Word2Vec(texts, 
                             vector_size=250, 
                             min_count=20, 
                             max_vocab_size=8000,
                             window=8,
                             epochs=5,
                             sg=0,
                             sample=1e-4,
                             hs=0,
                             negative=10,
                             ns_exponent=0.9,
                             cbow_mean=1
)

In [32]:
w2v.wv.most_similar('больница')

[('дом', 0.7946489453315735),
 ('школа', 0.75991290807724),
 ('московский', 0.7353457808494568),
 ('работать', 0.7282063364982605),
 ('открыть', 0.6969732046127319),
 ('училище', 0.6908789277076721),
 ('детский', 0.6648695468902588),
 ('1896', 0.6599502563476562),
 ('архитектор', 0.6496672630310059),
 ('сад', 0.6479186415672302)]

In [33]:
ft = gensim.models.FastText(texts, min_n=4, max_n=7)

In [34]:
ft.wv.most_similar('больница')

[('мельница', 0.8621236085891724),
 ('«дом', 0.8460416197776794),
 ('усыпальница', 0.8318303227424622),
 ('гостиница', 0.8172023296356201),
 ('дом', 0.8004085421562195),
 ('коласа', 0.7991678714752197),
 ('сад', 0.790465235710144),
 ('дом»', 0.7830639481544495),
 ('узедом', 0.7776100635528564),
 ('старица', 0.7747207880020142)]

# Задание 3 (4 балла)

Используя датасет для классификации (labeled.csv) и простую нейронную сеть (последняя модель в семинаре), оцените качество полученных эмбедингов в задании 1 и 2 (4 набора эмбедингов), также проверьте 1 любую из предобученных моделей с rus-vectores (но только не tayga_upos_skipgram_300_2_2019). 
Какая модель показывает наилучший результат?

Убедитесь, что для каждой модели вы корректно воспроизводите пайплайн предобработки (в 1 задании у вас лемматизация, не забудьте ее применить к датасету для классификации; у выбранной предобученной модели может быть своя специфичная предобработка - ее нужно воспроизвести)

In [35]:
data = pd.read_csv('drive/MyDrive/labeled.csv')
data['norm_text'] = data.comment.apply(preprocess)
data

,comment,toxic,norm_text
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[верблюд-то, за, что, дебил, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохол, это, отдушина, затюканый, россиянин, м..."
2,Собаке - собачья смерть\n,1.0,"[собака, , собачий, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страница, обновить, дебил, это, тоже, не, оск..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[ты, не, убедить, 6-страничный, пдф, в, тот, ч..."
...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,"[вонючий, совковый, скот, прибежать, и, ныть, ..."
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,"[а, кто, любить, гоблин, тупорылый, что-ли, ил..."
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,"[посмотреть, утомлённый, солнце, 2, и, оказать..."
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,"[крымотред, нарушать, правило, раздел, т.к, в,..."


In [36]:
vocab = Counter()

for text in data['norm_text']:
    vocab.update(text)
    
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 5:
        filtered_vocab.add(word)

len(filtered_vocab)

6309

In [37]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
id2word = {i:word for word, i in word2id.items()}

In [38]:
X = []

for tokens in data['norm_text']:
    ids = [word2id[token] for token in tokens if token in word2id]
    X.append(ids)

In [39]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=100)
y = data.toxic.values

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [41]:
weights = np.zeros((len(word2id), 100))

for word, i in word2id.items():
    if word == 'PAD':
        continue

    try:
        weights[i] = ft.wv[word]
    
    
    except KeyError:
        continue
        weights[i] = ft.wv['орлижльдвч']

In [42]:
inputs = tf.keras.layers.Input(shape=(100,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100, 
                                       trainable=False,
                                       weights=[weights])(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(embeddings)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=10)

Epoch 1/10
428/428 [==============================] - 2s 3ms/step - loss: 0.6304 - accuracy: 0.6517 - val_loss: 0.5876 - val_accuracy: 0.6865
Epoch 2/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6043 - accuracy: 0.6640 - val_loss: 0.5763 - val_accuracy: 0.6879
Epoch 3/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5938 - accuracy: 0.6666 - val_loss: 0.5731 - val_accuracy: 0.6893
Epoch 4/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5873 - accuracy: 0.6709 - val_loss: 0.5666 - val_accuracy: 0.6935
Epoch 5/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5823 - accuracy: 0.6754 - val_loss: 0.5636 - val_accuracy: 0.6990
Epoch 6/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5790 - accuracy: 0.6802 - val_loss: 0.5620 - val_accuracy: 0.7074
Epoch 7/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5763 - accuracy: 0.6817 - val_loss: 0.5607 - val_accuracy: 0.7046
Epoch 

In [44]:
weights = np.zeros((len(word2id), 250))

for word, i in word2id.items():
    if word == 'PAD':
        continue

    try:
        weights[i] = w2v.wv[word]
    
    
    except KeyError:
        continue
        weights[i] = np.random.rand(250)

In [45]:
inputs = tf.keras.layers.Input(shape=(100,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=250, 
                                       trainable=False,
                                       weights=[weights])(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(embeddings)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=10)

Epoch 1/10
428/428 [==============================] - 2s 3ms/step - loss: 0.6312 - accuracy: 0.6551 - val_loss: 0.5904 - val_accuracy: 0.6865
Epoch 2/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6034 - accuracy: 0.6640 - val_loss: 0.5777 - val_accuracy: 0.6865
Epoch 3/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5914 - accuracy: 0.6639 - val_loss: 0.5680 - val_accuracy: 0.6865
Epoch 4/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5831 - accuracy: 0.6651 - val_loss: 0.5614 - val_accuracy: 0.6865
Epoch 5/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5769 - accuracy: 0.6677 - val_loss: 0.5568 - val_accuracy: 0.6963
Epoch 6/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5720 - accuracy: 0.6716 - val_loss: 0.5525 - val_accuracy: 0.7032
Epoch 7/10
428/428 [==============================] - 1s 3ms/step - loss: 0.5682 - accuracy: 0.6750 - val_loss: 0.5506 - val_accuracy: 0.7060
Epoch 

In [51]:
weights = np.zeros((len(word2id), 300))

for word, i in word2id.items():
    if word == 'PAD':
        continue

    try:
        weights[i] = embeddings_sg[word2id[word]]
    
    
    except KeyError:
        continue
        weights[i] = np.random.rand(300)

In [52]:
inputs = tf.keras.layers.Input(shape=(100,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300, 
                                       trainable=False,
                                       weights=[weights])(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(embeddings)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [54]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=10)

Epoch 1/10
428/428 [==============================] - 2s 3ms/step - loss: 0.6477 - accuracy: 0.6598 - val_loss: 0.6095 - val_accuracy: 0.6865
Epoch 2/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6222 - accuracy: 0.6640 - val_loss: 0.6004 - val_accuracy: 0.6865
Epoch 3/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6199 - accuracy: 0.6640 - val_loss: 0.5983 - val_accuracy: 0.6865
Epoch 4/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6188 - accuracy: 0.6640 - val_loss: 0.5972 - val_accuracy: 0.6865
Epoch 5/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6177 - accuracy: 0.6640 - val_loss: 0.5961 - val_accuracy: 0.6865
Epoch 6/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6166 - accuracy: 0.6640 - val_loss: 0.5956 - val_accuracy: 0.6865
Epoch 7/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6156 - accuracy: 0.6640 - val_loss: 0.5948 - val_accuracy: 0.6865
Epoch 

In [56]:
weights = np.zeros((len(word2id), 300))

for word, i in word2id.items():
    if word == 'PAD':
        continue

    try:
        weights[i] = embeddings_cbow[word2id[word]]
    
    
    except KeyError:
        continue
        weights[i] = np.random.rand(300)

In [57]:
inputs = tf.keras.layers.Input(shape=(100,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300, 
                                       trainable=False,
                                       weights=[weights])(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(embeddings)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [58]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=10)

Epoch 1/10
428/428 [==============================] - 2s 5ms/step - loss: 0.6323 - accuracy: 0.6640 - val_loss: 0.5996 - val_accuracy: 0.6865
Epoch 2/10
428/428 [==============================] - 2s 4ms/step - loss: 0.6212 - accuracy: 0.6640 - val_loss: 0.5981 - val_accuracy: 0.6865
Epoch 3/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6196 - accuracy: 0.6640 - val_loss: 0.5969 - val_accuracy: 0.6865
Epoch 4/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6181 - accuracy: 0.6640 - val_loss: 0.5955 - val_accuracy: 0.6865
Epoch 5/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6165 - accuracy: 0.6640 - val_loss: 0.5952 - val_accuracy: 0.6865
Epoch 6/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6150 - accuracy: 0.6640 - val_loss: 0.5953 - val_accuracy: 0.6865
Epoch 7/10
428/428 [==============================] - 1s 3ms/step - loss: 0.6137 - accuracy: 0.6640 - val_loss: 0.5934 - val_accuracy: 0.6865
Epoch 

Лучшее качество на валидации у модели с эмбеддингами word2vec, модель с fasttext не сильно хуже. Модели с skipgram и cbow негативным семплированием ведут себя странно на валидации, но у меня не получилось понять почему. Тем не менее, видно, что лосс уменьшается, в этом плане модель cbow лучше по качеству, чем skipgram (как и казалось по результатам поиска ближайших слов)